# Partie1: Préparation des données pour un RAG chatbot.

Cette partie prépare la base de données et les embeddings à partir de fichiers texte.


Connexion à la Base de données 


In [213]:
import psycopg # On se connecte à PostgreSQL via psycopg. 

conn = psycopg.connect(
    dbname="mydb",
    user="admin",
    password="admin",
    host="127.0.0.1",
    port=5435
)
print("Connexion OK")


Connexion OK


Creer l'extension pgvector dans la base

 pgvector permet de stocker des vecteurs (embeddings) dans PostgreSQL et de faire des recherches de similarité.

In [214]:
with conn.cursor() as cur:
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    print("Extension pgvector créée")


Extension pgvector créée


Créer la table pour stocker les embeddings

In [208]:
# Charger le modèle de embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_dim = model.get_sentence_embedding_dimension() 


In [ ]:
with conn.cursor() as cur:
    #si je veux supprimer la table existante
    cur.execute("DROP TABLE IF EXISTS embeddings;")
    
    # Créer la table avec le paramètre embedding_dim
    cur.execute(f"""
        CREATE TABLE embeddings (
            id SERIAL PRIMARY KEY,
            corpus TEXT,
            embedding VECTOR({embedding_dim})
        );
    """)
    
    print("Table embeddings créée")


Lire et filtrer les fichiers texte


In [ ]:
from typing import List
def create_conversation_list(file_path: str) -> List[str]:
    encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
    for encoding in encodings:
        try:
            with open(file_path, "r", encoding=encoding) as file:
                text = file.read()
                if not text.strip():
                    print(f"Aucun contenu valide trouvé dans le fichier: {file_path}")
                    return []

                text_list = text.split("\n")
                filtered_list = [chaine.removeprefix("     ").strip() 
                                 for chaine in text_list if chaine.strip() and not chaine.startswith("<")]

                if not filtered_list:
                    print(f"Aucune ligne valide trouvée dans le fichier: {file_path}")
                else:
                    print(f"Lignes valides trouvées avec l'encodage {encoding}: {len(filtered_list)}")

                return filtered_list
        except UnicodeDecodeError:
            continue
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier {file_path} avec l'encodage {encoding}: {e}")
            return []

    print(f"Impossible de lire le fichier {file_path} avec les encodages testés.")
    return []

Calculer les embeddings d'une ligne de texte

In [ ]:
# Fonction pour calculer les embeddings avec SentenceTransformer
def calculate_embeddings(corpus: str) -> List[float]:
    return model.encode([corpus])[0].tolist()

Insérer un embedding dans la base

In [ ]:
def save_embedding(corpus: str, embedding: List[float], cursor):
    cursor.execute(
        "INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s)",
        (corpus, embedding)
    )

Rechercher les textes les plus similaires

In [ ]:
# Fonction pour trouver les textes similaires
def similar_corpus(input_corpus: str, top_k: int = 3) -> List[Tuple[int, str, float]]:
    input_embedding = calculate_embeddings(input_corpus)

    with conn.cursor() as cur:
        cur.execute("""
            SELECT id, corpus, embedding <=> %s AS distance
            FROM embeddings
            ORDER BY distance
            LIMIT %s;
        """, (input_embedding, top_k))
        results = cur.fetchall()
    return results

In [ ]:
file_paths = [
    "../data/TRANS_TXT/017_00000012.txt",
    "../data/TRANS_TXT/018_00000013.txt",
    "../data/TRANS_TXT/019_00000014.txt",
    "../data/TRANS_TXT/020_00000015.txt",
    "../data/TRANS_TXT/038_00000027.txt",
    ]       

 Charger tous les fichiers et insérer les embeddings

In [ ]:
import os
import time
# Charger le corpus depuis tous les fichiers
corpus_list = []
for path in file_paths:
    if os.path.exists(path):
        lines = create_conversation_list(path)
        corpus_list.extend(lines)
    else:
        print(f"Fichier introuvable et ignoré : {path}")

if not corpus_list:
    print("Aucune ligne valide n'a été chargée.")
else:
    print(f"{len(corpus_list)} lignes valides chargées depuis tous les fichiers")

# Insérer les embeddings dans la base de données
with conn.cursor() as cur:
    for i, corpus in enumerate(corpus_list):
        try:
            embedding = calculate_embeddings(corpus)
            save_embedding(corpus, embedding, cur)
            if i % 10 == 0:  # délai toutes les 10 lignes pour éviter surcharge
                time.sleep(1)
        except Exception as e:
            print(f"Erreur lors de la génération de l'embedding pour la ligne {i}: {e}")
    conn.commit()

print("Tous les embeddings ont été insérés dans la base de données")


Lignes valides trouvées avec l'encodage latin-1: 43
Lignes valides trouvées avec l'encodage latin-1: 9
Lignes valides trouvées avec l'encodage latin-1: 12
Lignes valides trouvées avec l'encodage latin-1: 13
Lignes valides trouvées avec l'encodage latin-1: 35
112 lignes valides chargées depuis tous les fichiers
Tous les embeddings ont été insérés dans la base de données


Vérification des premiers embeddings insérés

In [ ]:
with conn.cursor() as cur:
    cur.execute("SELECT id, corpus, embedding FROM embeddings LIMIT 10;")
    rows = cur.fetchall()
    for row in rows:
        print(f"ID: {row[0]}")
        print(f"Corpus: {row[1]}")
        print(f"Embedding (longueur {len(row[2])}): {row[2][:10]} ...")  # affiche les 10 premières valeurs
        print("---")


ID: 1
Corpus: h: U B S bonjour
Embedding (longueur 4740): [-0.055600 ...
---
ID: 2
Corpus: c: oui bonjour e j'appelle je sais pas si j'appelle au bon endroit e
Embedding (longueur 4694): [-0.051114 ...
---
ID: 3
Corpus: h: je vous écoute
Embedding (longueur 4715): [-0.031032 ...
---
ID: 4
Corpus: c: c'est pour
Embedding (longueur 4701): [-0.057875 ...
---
ID: 10
Corpus: h: oui
Embedding (longueur 4709): [-0.086280 ...
---
ID: 5
Corpus: c: e c'est pour savoir si la fac pendant l'été e a des professeurs ou des des gens qui font des stages de de perfectionnement en anglais et en espagnol
Embedding (longueur 4722): [-0.031055 ...
---
ID: 6
Corpus: h: e ce serait pour vous vous souhaiteriez
Embedding (longueur 4707): [-0.044720 ...
---
ID: 7
Corpus: h: non
Embedding (longueur 4711): [0.0024822 ...
---
ID: 8
Corpus: c: non non c'est pas pour moi
Embedding (longueur 4705): [-0.031244 ...
---
ID: 9
Corpus: c: ce serait pour ma fille
Embedding (longueur 4713): [-0.052667 ...
---


In [ ]:
conn.rollback()
print("Transaction PostgreSQL réinitialisée")


Transaction PostgreSQL réinitialisée


In [ ]:
#Cette version transforme la liste de floats en string pour que pgvector puisse comparer.
#Fonction de similarité utilisant pgvector sous forme de string

def similar_corpus(input_corpus: str, top_k: int = 3):
    # Embedding Python → liste de floats
    input_embedding = calculate_embeddings(input_corpus)

    # Transformer en format pgvector "[0.1, 0.2, ...]"
    vector_str = "[" + ",".join(map(str, input_embedding)) + "]"

    with conn.cursor() as cur:
        cur.execute("""
            SELECT id, corpus, embedding <=> %s AS distance
            FROM embeddings
            ORDER BY distance
            LIMIT %s;
        """, (vector_str, top_k))

        return cur.fetchall()


Test de similarité

In [ ]:
test_text = "Bonjour, j’aimerais savoir où se trouve la réunion de ce soir."
results = similar_corpus(test_text, top_k=3)

for r in results:
    print(f"ID: {r[0]}")
    print(f"Corpus: {r[1]}")
    print(f"Distance: {r[2]}")
    print("---")


ID: 219
Corpus: c: e j'aimerais savoir e j'ai une réunion ce soir e
Distance: 0.16857539602664584
---
ID: 227
Corpus: h: oui c'est rue de la Loi la réunion
Distance: 0.35858527668802165
---
ID: 204
Corpus: h: ce soir il doit y avoir une réunion sur le D A E U qui doit se faire à l'I U(P) e rue de la Loi
Distance: 0.38081280107421756
---
